In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 32
VAL_SIZE = 0.2
AMNT_BATCHS = 400
DATASET_SIZE = AMNT_BATCHS * BATCH_SIZE
EPOCHS = 40
NRO_MODELO = "modelo_NoAERandomPosta"

In [ ]:
trainImages = keras.utils.image_dataset_from_directory(
    'places/',
    label_mode=None,
    image_size=(256,256),
    batch_size=BATCH_SIZE,
    shuffle=False).take(AMNT_BATCHS)

trainImages = trainImages.map( lambda x : x / 255 )

blurredTrainImages = keras.utils.image_dataset_from_directory(
    'blurredRandomPlaces/',
    label_mode=None,
    image_size=(256,256),
    batch_size=BATCH_SIZE,
    shuffle=False).take(AMNT_BATCHS)

blurredTrainImages = blurredTrainImages.map( lambda x : x / 255 )

In [ ]:
valSplit = int( (DATASET_SIZE / BATCH_SIZE) * VAL_SIZE )

train =  tf.data.Dataset.zip((blurredTrainImages, trainImages))

datasetTrain = train.skip(valSplit)
datasetValidation = train.take(valSplit)

In [ ]:
input = layers.Input(shape=(256, 256, 3))

# Encoder
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(input)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(32, (5, 5), activation="relu", padding="same")(x)
x = layers.Conv2D(3, (5, 5), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

modelFile = open('checkpoints/' + NRO_MODELO + '/modelo.txt', 'w+')
autoencoder.summary(print_fn=lambda x: modelFile.write(x + '\n'))
modelFile.close()

In [ ]:
def writeLog( file, epoch, log ):
    file.write(str(epoch) + ',' + str(log['loss'])+ ',' + str(log['val_loss']) + '\n')


In [ ]:
modelCallback = tf.keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/' + NRO_MODELO + '/model.ckpt',
        save_weights_only=True,
        verbose=1)

lossCkptFile = open('checkpoints/' + NRO_MODELO + '/loss.ckpt', 'a+')
lossCallback = keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: writeLog( lossCkptFile, epoch, logs))

historia = autoencoder.fit(
    datasetTrain,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=datasetValidation,
    callbacks = [modelCallback, lossCallback]
)

lossCkptFile.close()

In [ ]:
predictions = autoencoder.predict( datasetValidation.take(1) )

In [ ]:
def plotPrediccion( entrada, prediccion ):
    plt.figure(figsize=(20, 20))
    for i, (image1, image2) in enumerate(zip(entrada, prediccion)):
        ax = plt.subplot(2, 1, i + 1)
        image1Np = image1[0].numpy()
        #clip01 = lambda x : (x + np.ones_like(x) * np.abs( x.min() ) ) / (x + np.ones_like(x) * np.abs( x.min() ) ).max()
        
        plt.imshow( image1Np[0] )

        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, 1, i + 2)

        plt.imshow( image2 )

        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
plotPrediccion( datasetValidation.take(1), predictions)

In [ ]:
plt.plot(historia.history['loss'])
plt.plot(historia.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()